<a href="https://colab.research.google.com/github/Jose-Augusto-C-M/KettleHole_SLIC_CNN/blob/main/Multiclass_Unet_3backbones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

In [2]:
!pip install segmentation-models

In [3]:
!cp /content/drive/MyDrive/datasets/projeto_marlene/simple_multi_unet_model.py /content

In [4]:
from simple_multi_unet_model import multi_unet_model
import tensorflow as tf

In [5]:
from tensorflow.keras.utils import normalize
import os
import glob
import cv2
import numpy as np
from matplotlib import pyplot as plt
import segmentation_models as sm

from tensorflow.keras.utils import normalize
from tensorflow.keras.metrics import MeanIoU

Segmentation Models: using `keras` framework.


In [6]:
SIZE_X = 512
SIZE_Y = 512
n_classes = 17

In [7]:
train_images = []

for directory_path in glob.glob('/content/drive/MyDrive/datasets/projeto_marlene/DL_train/ImageOutTif'):
    for img_path in glob.glob(os.path.join(directory_path, '*.tif')):
        img = cv2.imread(img_path, 1)
        train_images.append(img)

train_images = np.array(train_images)

train_masks = []

for directory_path in glob.glob('/content/drive/MyDrive/datasets/projeto_marlene/DL_train/MaskOutTif'):
    for mask_path in glob.glob(os.path.join(directory_path, '*.tif')):
        mask = cv2.imread(mask_path,-1)
        train_masks.append(mask)

train_masks = np.array(train_masks)

In [8]:
np.shape(train_images)

(644, 256, 256, 3)

In [9]:
np.shape(train_masks)

(644, 256, 256)

In [10]:
np.unique(train_masks)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16],
      dtype=uint16)

In [11]:
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()

n, h, w = train_masks.shape
train_masks_reshaped = train_masks.reshape(-1,1)
train_masks_reshaped_encoded = labelencoder.fit_transform(train_masks_reshaped)
train_masks_encoded_original_shape = train_masks_reshaped_encoded.reshape(n, h, w)

np.unique(train_masks_encoded_original_shape)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16])

In [12]:
train_masks_input = np.expand_dims(train_masks_encoded_original_shape, axis=3)

train_masks_reshaped_encoded = np.expand_dims(train_masks_reshaped_encoded, axis = 1)  
np.shape(train_masks_reshaped_encoded)

np.unique(train_masks)

In [19]:
# train_images = np.expand_dims(train_images, axis=3)
# train_images = normalize(train_images, axis=1)

#Create a subset of data for quick testing
#Picking 10% for testing and remaining for training

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_images, train_masks_input, test_size = 0.10, random_state = 0)

#Further split training data t a smaller subset for quick testing of models
# X_train, X_do_not_use, y_train, y_do_not_use = train_test_split(X1, y1, test_size = 0.2, random_state = 0)

print("Class values in the dataset are ... ", np.unique(y_train))  # 0 is the background/few unlabeled 

Class values in the dataset are ...  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]


np.shape(X_train)

np.unique(train_images)

In [20]:
from tensorflow.keras.utils import to_categorical
train_masks_cat = to_categorical(y_train, num_classes=n_classes)
y_train_cat = train_masks_cat.reshape((y_train.shape[0], y_train.shape[1], y_train.shape[2], n_classes))

test_masks_cat = to_categorical(y_test, num_classes=n_classes)
y_test_cat = test_masks_cat.reshape((y_test.shape[0], y_test.shape[1], y_test.shape[2], n_classes))

In [ ]:
np.shape(train_masks_cat)

In [ ]:
np.shape(y_train_cat)

In [ ]:
np.shape(test_masks_cat)

In [ ]:
np.shape(y_test_cat)

In [21]:
sm.set_framework('tf.keras')

sm.framework()

'tf.keras'

In [22]:
n_classes=17
activation='softmax'

LR = 0.0001
optim = tf.keras.optimizers.Adam(LR)

# Segmentation models losses can be combined together by '+' and scaled by integer or float factor
# set class weights for dice_loss (car: 1.; pedestrian: 2.; background: 0.5;)
dice_loss = sm.losses.DiceLoss(class_weights=np.array([0.058, 0.058,0.058,0.058,0.058,0.058,0.058,0.058,0.058,0.058,0.058,0.058,0.058,0.058,0.058,0.058])) 
focal_loss = sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

# actulally total_loss can be imported directly from library, above example just show you how to manipulate with losses
# total_loss = sm.losses.binary_focal_dice_loss # or sm.losses.categorical_focal_dice_loss 

metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

In [ ]:
np.shape(X_train)

In [ ]:
BACKBONE1 = 'resnet34'
preprocess_input1 = sm.get_preprocessing(BACKBONE1)

# preprocess input
X_train1 = preprocess_input1(X_train)
X_test1 = preprocess_input1(X_test)

# define model
model1 = sm.Unet(BACKBONE1, encoder_weights='imagenet', classes=n_classes, activation=activation)

# compile keras model with defined optimozer, loss and metrics
#model1.compile(optim, total_loss, metrics=metrics)

model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=metrics)

# print(model1.summary())


history1=model1.fit(X_train1, 
          y_train_cat,
          batch_size=8, 
          epochs=50,
          verbose=1,
          validation_data=(X_test1, y_test_cat))


model1.save('res34_backbone_50epochs.hdf5')

Epoch 1/50
73/73 [==============================] - 762s 10s/step - loss: 2.1839 - iou_score: 0.5646 - f1-score: 0.5649 - val_loss: 4.3384 - val_iou_score: 0.5855 - val_f1-score: 0.5918
Epoch 2/50
73/73 [==============================] - 757s 10s/step - loss: 1.9665 - iou_score: 0.5863 - f1-score: 0.5880 - val_loss: 36378.6797 - val_iou_score: 0.5605 - val_f1-score: 0.5698
Epoch 3/50
26/73 [=========>....................] - ETA: 7:57 - loss: 1.9991 - iou_score: 0.5633 - f1-score: 0.5633

In [ ]:
###Model 2

BACKBONE2 = 'inceptionv3'
preprocess_input2 = sm.get_preprocessing(BACKBONE2)

# preprocess input
X_train2 = preprocess_input2(X_train)
X_test2 = preprocess_input2(X_test)

# define model
model2 = sm.Unet(BACKBONE2, encoder_weights='imagenet', classes=n_classes, activation=activation)


# compile keras model with defined optimozer, loss and metrics
model2.compile(optim, total_loss, metrics)
#model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=metrics)


print(model2.summary())


history2=model2.fit(X_train2, 
          y_train_cat,
          batch_size=8, 
          epochs=50,
          verbose=1,
          validation_data=(X_test2, y_test_cat))


model2.save('inceptionv3_backbone_50epochs.hdf5')

In [ ]:
BACKBONE3 = 'vgg16'
preprocess_input3 = sm.get_preprocessing(BACKBONE3)

# preprocess input
X_train3 = preprocess_input3(X_train)
X_test3 = preprocess_input3(X_test)


# define model
model3 = sm.Unet(BACKBONE3, encoder_weights='imagenet', classes=n_classes, activation=activation)

# compile keras model with defined optimozer, loss and metrics
model3.compile(optim, total_loss, metrics)
#model3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=metrics)


print(model3.summary())

history3=model3.fit(X_train3, 
          y_train_cat,
          batch_size=8, 
          epochs=50,
          verbose=1,
          validation_data=(X_test3, y_test_cat))


model3.save('vgg19_backbone_50epochs.hdf5')

In [ ]:
#plot the training and validation accuracy and loss at each epoch
loss = history1.history['loss']
val_loss = history1.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

acc = history1.history['iou_score']
val_acc = history1.history['val_iou_score']

plt.plot(epochs, acc, 'y', label='Training IOU')
plt.plot(epochs, val_acc, 'r', label='Validation IOU')
plt.title('Training and validation IOU')
plt.xlabel('Epochs')
plt.ylabel('IOU')
plt.legend()
plt.show()


from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', 
                                                  classes = np.unique(train_masks_reshaped_encoded), 
                                                  y = train_masks_reshaped_encoded
                                                 )
print("Class weights are...:", class_weights)

from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight( "balanced", np.unique(train_masks_reshaped_encoded), train_masks_reshaped_encoded)
class_weights = dict(zip(np.unique(train_masks_reshaped_encoded), class_weights))
class_weights

IMG_HEIGHT = X_train.shape[1]
IMG_WIDTH  = X_train.shape[2]
IMG_CHANNELS = X_train.shape[3]

def get_model():
    return multi_unet_model(n_classes=n_classes, IMG_HEIGHT=IMG_HEIGHT, IMG_WIDTH=IMG_WIDTH, IMG_CHANNELS=IMG_CHANNELS)

model = get_model()
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer='adam',loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
model.summary()        

np.shape(X_train)

np.shape(y_train_cat)

newInput = tf.keras.layers.Input(batch_shape=(0,199,199,3))
op = model(newInput)

#If starting with pre-trained weights. 
#model.load_weights('???.hdf5')

history = model.fit(X_train, y_train_cat, 
                    batch_size = 16, 
                    verbose=1, 
                    epochs=50, 
                    validation_data=(X_test, y_test_cat), 
                    #class_weight=class_weights,
                    shuffle=False)